# Set Up

In [1]:
#The Following cell of code is used everytime FASTAI library is used.
#They tell the notebook to reload any changes made to any libraries used.
#They also ensure that any graphs are plotted are shown in this notebook
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision.all import *
from fastai.metrics import *
from pathlib import Path

In [3]:
!pip install wwf timm -qqq
#downgrade pytorch because 1.9.1 is causing conflicts
!pip install --user torch==1.9.0

In [4]:
path = Path('../input/fancy-watche-images/content/images')

In [5]:
Path.BASE_PATH = path
path.ls()

we have 10 brands **'Breitling', 'Tissot', 'Omega', 'Rolex', 'Audemars Piguet', 'Breguet', 'Maurice Lacroix', 'Rado', 'Patek Philippe', 'Vacheron Constantin'**

* cool! we got 200 image for each watch brand

# Constructing a DataBlock

In [6]:
path = Path('../input/fancy-watche-images/content/images')
watches = DataBlock(
    # Designation the independent and dependent variables
    blocks = (ImageBlock, CategoryBlock), 
    # To get a list of those files,and returns a list of all of the images in that path
    get_items = get_image_files, 
    # Split our training and validation sets randomly
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    # We are telling fastai what function to call to create the labels in our dataset, in our case is independet variable
    get_y = parent_label,
    # Will resize these images to the same size
    item_tfms = Resize(128))

dls = watches.dataloaders(path,bs=128) # bs = batch size

# Display the images

In [7]:
dls.valid.show_batch(max_n = 4, nrows = 1)

# Resizing and Data augmentation

We don't have a lot of data for our problem (200 pictures of each sort of bear at most), so to train our model, we'll use RandomResizedCrop with an image size of 224 px, which is fairly standard for image classification, and default aug_transforms:

In [8]:
watches = watches.new(
    item_tfms = RandomResizedCrop(224, min_scale = 0.5),
    batch_tfms = aug_transforms()
)
dls = watches.dataloaders(path)

In [9]:
dls.show_batch()

# Train a simple model

In [10]:
learn = vision_learner(dls,
                    resnet18,
                    metrics=accuracy)
learn.fine_tune(3)

# Find a good learning rate

In [11]:
lr_min,lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))

In [12]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [13]:
learn = cnn_learner(dls,
                    resnet34,
                    metrics=accuracy)
learn.fit_one_cycle(4,1e-3)

# Unfreezing and Transfer Learning

In [14]:
learn.unfreeze()

In [15]:
lr_min,lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))

In [16]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [17]:
learn.fit_one_cycle(30, lr_max=slice(1e-3,1e-2))

# Deeper Architectures

The other downside of deeper architectures is that they take quite a bit longer to train. One technique that can speed things up a lot is *mixed-precision training*. This refers to using less-precise numbers (*half-precision floating point*, also called *fp16*) where possible during training. nearly all current NVIDIA GPUs support a special feature called *tensor cores* that can dramatically speed up neural network training, by 2-3x. They also require a lot less GPU memory. To enable this feature in fastai, just add `to_fp16()` after your `Learner` creation.

**using fp16 training. In particular,  that fp16 training can sometimes lead to poorer model accuracy (due to the added randomness), and that it's important to keep an eye on the model's performance when using this technique.**

You can't really know ahead of time what the best architecture for your particular problem is—you need to try training some. So let's try a ResNet-50 now with mixed precision:

In [18]:
learner = vision_learner(dls,
                    resnet50,
                    metrics=accuracy).to_fp16()
#learner.fit_one_cycle(4)
#learner.unfreeze()
#learner.fit_one_cycle(40,lr_max=slice(1e-3,1e-2))
learner.fine_tune(40,freeze_epochs=3)

* As we can see that it's not always better to have deeper architectures to get high accuracy.
* As we saw that a resnet34 got us about 82.2% accuracy while resnet50 achived just 84.5%.

# Save the Model

# Model Interpretation

In [19]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_confusion_matrix(figsize=(8,8), dpi=60)

In [20]:
learn.export()

In [21]:
path = Path()
path.ls(file_exts='.pkl')

In [22]:
learn_inf = load_learner(path/'export.pkl')

In [23]:
learn_inf.dls.vocab

# Creating a Notebook App from the Model

In [26]:
from fastai.vision.widgets import *
btn_upload = widgets.FileUpload()
btn_upload

In [27]:
btn_upload = SimpleNamespace(data = ['../input/fancy-watche-images/content/images/Rolex/001_198f48ca.jpg'])

In [28]:
img = PILImage.create(btn_upload.data[-1])

We can use an Output widget to display it:

In [29]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [30]:
pred,pred_idx,probs = learn_inf.predict(img)

and use a Label to display them:

In [31]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

We'll need a button to do the classification. It looks exactly like the upload button:

In [32]:
btn_run = widgets.Button(description='Classify')
btn_run

We'll also need a click event handler; that is, a function that will be called when it's pressed. We can just copy over the lines of code from above:

In [33]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [34]:
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [35]:
#hide_output
VBox([widgets.Label('Select your leopard!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

#### I hope you like it